# LLM Agent를 만드는 여정 (1) LLM의 답변을 정형화하기

## 들어가기에 앞서 

프로그래머로서, GPT o1은 충격적이었습니다. GPT o1와 [Cursor](https://www.cursor.com/)의 사용법만 배운 채로, 기획자 친구가 1시간도 안되는 시간 안에 IOS 앱을 빌드하고 실행하는 것을 성공해냈습니다. 이 모습을 보며 "프로그램을 작성한다"라는 일은 몇년 내로 "엑셀 다룰 줄 알아요"같은, 그것만으로는 먹고 살지 못하는 능력으로 자리잡을 것이라 생각이 들었습니다. 친구와 저는 카페에서 "이제 개발하는 것은 옛날 계산기 나오기 전에 주판을 튀기는 것과 같은 거야"라고 얘기 나눴습니다.

지금, 당장, 가장 필요한 일은 무엇일까. 저는 GPT의 잠재력을 끌어내기 위해, GPT에게 눈(Perception)과 손(Action)을 달아주는 것이라 생각했습니다. 

지금, 당장, 가장, 필요한 기술적 역량은 "GPT의 강력한 사고하는 능력을 어떻게 끌어낼 것인가"이라 바라보고 있습니다. 단순히 Text를 주고 받는 LLM에서 벗어나서, 직접 행동으로 옮길 수 있는 능력을 만들어내야 합니다. GPT가 자율적으로 우리를 도와줄 수 있는 시스템, 즉 AI Agent를 만들어 내는 것이 필수라 생각했습니다. 지금 글을 쓰는 이 순간에는 어떻게 만들어야 할지 잘 모르겠어요. 만들어 가는 과정에서 얻은 지식들을 하나씩 정리하고자 합니다.

### 시리즈 목표 : 나를 위한 AI Agent 만들기

<img src="../resources/assets/huggingface.png" width="50%"/>

구체적으로는 좀 더 자극적이게 "나에게 필요한 AI 서비스를 찾고, 만들어주는 AI Agent"를 만들어 보고자 합니다. 대표적인 AI 모델 플랫폼인 [Huggingface](https://huggingface.co/)는 2024년 10월 기준 99만개에 달하는 AI 모델이 존재합니다. 하지만 여기에서 우리에게 딱 필요한 모델을 찾는 것은 쉽지 않습니다. 여기서 필요한 모델을 찾아, 우리에게 필요한 형태로 가공하는 역할을 수행하는 AI 서비스를 만들어 보고자 합니다.

### 구축 환경

저에게 친숙하고 LLM 생태계에서도 가장 널리 사용하는 도구들을 선택했습니다.

- Python3 
- Jupyter Notebook
- Langchain

## 오늘의 목표 : LLM의 TEXT 답변을 정형화하기 


이 시리즈의 첫 번째 글로, 프롬프트 템플릿(PromptTemplate)과 출력 파서(OutputParser)을 통해, 어떻게 Langchain으로 프롬프트 개발을 편리하게 할 수 있는지를 설명하겠습니다.

LLM은 단순히 Text를 반환하는 함수입니다. 이런 LLM이 저 대신 카카오T를 부르고, 쿠팡에서 주문하고, 코드를 작성하기 위해서는, Text가 아니라 Action을 수행하는 기능들을 호출할 수 있어야 합니다. 이를 위해서 LLM이 비정형화된 Text 대신 Text를 반환하도록 만들어 두어야 합니다. 

## 환경설정

<img src="../resources/assets/langchain.png" width="50%">

Langchain은 LLM을 활용한 애플리케이션 개발을 지원하는 강력한 프레임워크입니다. OpenAI, Anthropic, Azure, Google, Groq 등 다양한 LLM과 호환되며, 각 모델 별로 추가 라이브러리를 설치해야 합니다.

먼저, Langchain과 Langchain OpenAI 라이브러리를 설치해보겠습니다.


In [ ]:
%%capture
!pip install langchain langchain_openai

#### OpenAI API KEY 발급받기

<img src="../resources/assets/openai-platform.png" width="30%">

[OpenAI Platform:api-key](https://platform.openai.com/api-keys)에서 API key를 발급받을 수 있습니다. 아래 스크립트를 실행하면, `getpass.getpass`를 사용하여 API 키를 안전하게 입력받아서 `ChatOpenAI` 객체를 초기화합니다. ChatOpenAI은 우리가 사용하는 ChatGPT의 대화창을 API로 호출할 수 있도록 만든 객체입니다.

In [1]:
import getpass
from langchain_openai import ChatOpenAI

api_key = getpass.getpass("openai api-key:")
llm = ChatOpenAI(api_key=api_key)

openai api-key: ········


## Langchain을 사용한 LLM 호출

프로그래밍 관점에서 LLM은 텍스트 메시지를 입력받아 텍스트 메시지를 출력하는 함수입니다. 

````mermaid
flowchart LR
    input[입력:\n안녕 LLM] == LLM ==> output[출력:\n반가워 휴먼]
````

간단하게 "안녕"이라고 입력하면, 아래와 같이 ChatGPT를 호출 후, AIMessage를 반환합니다. 

AIMessage는 말그대로 AI가 만든 Message로, LLM의 출력 메시지를 뜻합니다. LLM에서는 AIMessage 외에도, SystemMessage와 HumanMessage가 더 있습니다.

In [2]:
llm.invoke("안녕")

AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-83399f6a-119b-4e84-810b-2e37954f159c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32})

### 메시지 타입 이해: SystemMessage, HumanMessage, AIMessage

LLM에서 메시지 타입은 크게 3가지 유형으로 나뉩니다.

| 유형 | 역할 | 설명 |
| --- | --- | --- |
| SystemMessage | 대화의 맥락을 설정하고, 모델의 행동 지침을 제공 | 모델에게 특정 역할을 부여하거나, 대화의 규칙을 설정하는 데 사용  |
| HumanMessage | 사용자가 모델에게 전달하는 입력 | 사용자가 모델에게 질문을 하거나 요청을 전달하는 데 사용 |
| AIMessage | 모델이 생성한 응답 | 모델이 사용자에게 응답을 제공하는 데 사용 |

### SystemMessage과 HumanMessage

SystemMessage를 통해, AI에게 특정한 역할과 행동 지침을 부여할 수 있습니다. 예를 들어, AI가 유저의 엄격한 도덕 선생님 역할을 맡도록 설정할 수 있습니다. 이렇게 만든 규칙 하에서 우리의 요청은 HumanMessage로 정의 후 호출하면 됩니다.

In [3]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

system_message = SystemMessage(
    content="당신은 유저의 엄격한 도덕 선생님입니다. 유저가 높임말을 쓰도록 훈육해 주세요"
)
input_message = HumanMessage(content="안녕")

llm.invoke([
    system_message,
    input_message
])

AIMessage(content='안녕하세요. 높임말을 사용해 주세요. "안녕하세요"라고 써주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 64, 'total_tokens': 96, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0ab8a495-a230-428a-8eca-05bfb017c9d1-0', usage_metadata={'input_tokens': 64, 'output_tokens': 32, 'total_tokens': 96})

같은 인삿말이라도 SystemMessage이 어떻게 정했느냐에 따라, LLM은 서로 다르게 동작합니다. 이처럼, SystemMessage를 통해 AI의 역할과 응답 방식을 다양하게 설정할 수 있습니다.

In [4]:
system_message = SystemMessage(
    content="당신은 유저의 동갑내기 친구야. 유저에게 반갑게 화답해줘."
)

input_message = HumanMessage(content="안녕")

llm.invoke([
    system_message,
    input_message
])

AIMessage(content='안녕! 반가워 😊 어떤 이야기를 나누고 싶어?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 52, 'total_tokens': 81, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3c7acd61-87b5-457f-87b4-b342238c7228-0', usage_metadata={'input_tokens': 52, 'output_tokens': 29, 'total_tokens': 81})

## 프롬프트를 템플릿화하기 

프롬프트를 템플릿화하면 반복적인 작업을 쉽게 처리할 수 있습니다. **PromptTemplate**을 사용하면 프롬프트 내의 특정 부분을 변수로 대체하여 다양한 상황에 유연하게 대응할 수 있습니다. 이는 코드의 재사용성을 높이고, 유지보수를 용이하게 하며, 일관된 응답을 생성하는 데 큰 장점이 있습니다.

### PromptTemplate의 장점

- **재사용성**: 동일한 구조의 프롬프트를 여러 번 사용할 수 있어 효율적입니다.
- **유연성**: 변수 부분을 통해 다양한 입력에 대응할 수 있습니다.
- **일관성**: 일관된 프롬프트 구조를 유지하여 예측 가능한 AI 응답을 얻을 수 있습니다.

다음은 프롬프트 템플릿을 구성하는 예시입니다.

아래와 같은 프롬프트에서, SystemMessage에서 역할을 의미하는 `동갑내기 친구`와 HumanMessage에서의 유저 말만 매번 바뀐다고 생각해봅시다.

````
[system]
당신은 유저의 동갑내기 친구야. 그 역할에 따라 대화해주세요.

[human]
안녕, 지금 뭐해?
````

In [5]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

system_message = SystemMessagePromptTemplate.from_template("당신은 유저의 {role}입니다.")
human_message = HumanMessagePromptTemplate.from_template("{message}")

input_template = ChatPromptTemplate(
    messages=[system_message, human_message],
    input_variables=["role", "message"]
)

이렇게 만든 템플릿으로 아래와 같이, 실제 프롬프트를 생성할 수 있습니다.

In [6]:
print(input_template.format(role="동갑내기 친구", message="반가워"))

System: 당신은 유저의 동갑내기 친구입니다.
Human: 반가워


In [7]:
print(input_template.format(role="스승님", message="숙제 안해왔어요.."))

System: 당신은 유저의 스승님입니다.
Human: 숙제 안해왔어요..


## Langchain에서의 Chain

Langchain의 **Chain**을 활용하면, 프롬프트 생성에서부터 LLM 호출까지의 일련의 과정을 하나의 파이프라인으로 연결할 수 있습니다. 이를 통해 코드의 가독성과 유지보수성을 크게 향상시킬 수 있습니다. Chain을 사용하면 각 단계를 독립적으로 관리할 수 있으며, 필요에 따라 쉽게 확장할 수 있습니다.


In [8]:
chain = (
    input_template
    | llm
)

In [9]:
chain.invoke({'role': "선생님", "message":"숙제 안해왔어요.."})

AIMessage(content='숙제를 왜 안했나요? 어떤 이유가 있나요? 함께 이유를 알아보고 해결책을 찾아볼까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 37, 'total_tokens': 92, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-184ca0a0-2250-42bf-b9c3-d8596d46ec96-0', usage_metadata={'input_tokens': 37, 'output_tokens': 55, 'total_tokens': 92})

위와 같이 작성하면 크게 아래와 같은 장점이 있습니다.

- **가독성**: 복잡한 로직을 단순화하여 읽기 쉽게 만듭니다.
- **모듈화**: 각 단계를 독립적으로 관리할 수 있어 유지보수가 용이합니다.
- **확장성**: 새로운 기능을 쉽게 추가할 수 있습니다.


## LLM의 어려움: 다루기 어려운 텍스트

자연어는 인간이 이해하기 쉬운 형태이지만, 프로그래밍을 작성하는 입장에서는 다루기가 어렵습니다. 예를 들어 단순한 수학 계산을 요청하는 상황을 봅시다.

In [10]:
input_message = HumanMessage(content="x=1, y=2일 때, 2 * (3 * x + y * 5)는 얼마일까요?")

In [11]:
output0 = llm.invoke([input_message])
output0.content

'x=1, y=2일 때, 2 * (3 * x + y * 5)는 다음과 같이 계산할 수 있습니다.\n\n2 * (3 * 1 + 2 * 5)\n= 2 * (3 + 10)\n= 2 * 13\n= 26\n\n따라서, 2 * (3 * x + y * 5)는 26입니다.'

In [12]:
output1 = llm.invoke([input_message])
output1.content

'x = 1, y = 2를 대입하면,\n\n2 * (3 * 1 + 2 * 5) = 2 * (3 + 10) = 2 * 13 = 26\n\n따라서, 2 * (3 * x + y * 5)는 26입니다.'

이렇게 LLM은 다양한 형태로 응답합니다. 이런 게 다양한 형태의 텍스트에서 우리가 원하는 정답을 도출하는 것은 매우 까다롭습니다.  

### 응답값을 구조화하기

LLM이 반환하는 다양한 형태의 텍스트 응답을 효과적으로 처리하기 위해, 응답값을 구조화할 필요가 있습니다. 예를 들어, 단순한 수학 계산을 요청할 때, LLM이 JSON 형식으로 응답하도록 지시할 수 있습니다.

예를들어, chatGPT가 아래와 같이 응답한다면 어떨까요? 

````json
{
   "result": 26,
   "description":"2*(3 * x + y * 5) = 2*(3*1 + 2*5) = 2*(3 + 10) = 2*13 = 26\n\n따라서, 2*(3 * x + y * 5)의 값은 26입니다." 
}
````

우리는 훨씬 더 편하게 값을 처리할 수 있을 것입니다. 이렇게 출력값이 나오기 위해서는, 우리는 LLM에게 **어떻게 출력해야 하는지를** 지시해야 합니다. 
보통 아래와 같은 지시문을 `SystemMessage`에 담아서 호출합니다.

````
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"result": float  // The numerical result of the calculation
	"description": string  //description of the calculation process
}
```
````

In [13]:
instruction = (
'The output should be a markdown code snippet formatted in the following schema,' 
'including the leading and trailing "```json" and "```":\n\n'
'```json\n'
'{\n'
'	"description": string // description of the calculation process \n'
'	"result": float   // The numerical result of the calculation \n'    
"}\n"
"```\n"
)

input_message = HumanMessage(content="x=1, y=2일 때, 2 * (3 * x + y * 5)는 얼마일까요?")
output = llm.invoke([SystemMessage(instruction), input_message])
print(output.content)

```json
{
	"description": "계산 과정: 2 * (3 * 1 + 2 * 5) = 2 * (3 + 10) = 2 * 13 = 26",
	"result": 26
}
```


### OutputParser을 활용한 파싱

Langchain의 `StructuredOutputParser`를 사용하면, 원하는 형식의 응답을 쉽게 파싱할 수 있습니다. 이를 통해 응답 데이터를 체계적으로 처리하고 활용할 수 있습니다.

다음은 `StructuredOutputParser`를 설정하는 예시입니다.

In [14]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="result", description="The numerical result of the calculation", type='float'),
    ResponseSchema(name="description", description="description of the calculation process", type='string'),
]

output_parser = StructuredOutputParser(response_schemas=response_schemas)
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='result', description='The numerical result of the calculation', type='float'), ResponseSchema(name='description', description='description of the calculation process', type='string')])

파서에서 원하는 형태의 응답값을 받기 위한 지침은 아래와 같이 생성할 수 있습니다.

In [15]:
inst = output_parser.get_format_instructions()
print(inst)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"result": float  // The numerical result of the calculation
	"description": string  // description of the calculation process
}
```


이제 이 지침을 SystemMessage에 담아 LLM에게 전달하면, 구조화된 응답을 받을 수 있습니다.

In [16]:
system_message = SystemMessage(content=inst)
human_message = HumanMessagePromptTemplate.from_template("{message}")

input_template = ChatPromptTemplate(
    messages=[system_message, human_message],
    input_variables=["message"]
)

In [17]:
output = (
    input_template
    | llm
).invoke({"message": "x=1, y=2일 때, 2 * (3 * x + y * 5)는 얼마일까요?"})
print(output.content)

```json
{
	"result": 31.0,
	"description": "계산 과정: 2 * (3 * 1 + 2 * 5) = 2 * (3 + 10) = 2 * 13 = 26"
}
```


`StructuredOutputParser`를 사용하여 응답을 파싱하면, Dictionary 형태로 응답을 받을 수 있습니다.

In [18]:
r = output_parser.invoke(output.content)
r

{'result': 31.0,
 'description': '계산 과정: 2 * (3 * 1 + 2 * 5) = 2 * (3 + 10) = 2 * 13 = 26'}

output_parser도 Langchain의 chain을 통해 아래와 같이 처리할 수 있습니다.

In [19]:
(
    input_template
    | llm
    | output_parser
).invoke({"message": "x=1, y=2일 때, 2 * (3 * x + y * 5)는 얼마일까요?"})

{'result': 29.0,
 'description': '계산 순서에 따라 (3 * 1 + 2 * 5) = 13이 되고, 이후 2 * 13 = 26을 계산하면 최종 결과는 29.0이 됩니다.'}


### 마무리

이번 글에서는 Langchain과 LLM을 활용하여 AI 에이전트를 개발하는 과정을 살펴보았습니다. 프롬프트 템플릿화, 메시지 타입 이해, 출력 파서를 통한 응답값 구조화 등을 통해 보다 효과적이고 구조화된 AI 에이전트를 만드는 방법을 배웠습니다. 